* Install `pcdet` library and then generate the data infos by running the following command: 
```python 
python -m pcdet.datasets.kitti.kitti_dataset create_kitti_infos tools/cfgs/dataset_configs/kitti_dataset.yaml
```

In [1]:
from tqdm import tqdm

In [2]:
import sys
import yaml
from pathlib import Path
from easydict import EasyDict
import numpy as np

from pcdet.datasets import KittiDataset

import pandas_profiling
import pandas as pd

In [3]:
dataset_cfg_dir = '/home/farzad/OpenPCDet/tools/cfgs/dataset_configs/kitti_dataset.yaml'
pcdet_root_dir = Path('/home/farzad/OpenPCDet')
class_names = ['Pedestrian', 'Car']

In [4]:
dataset_cfg = EasyDict(yaml.load(open(dataset_cfg_dir)))
data_path = pcdet_root_dir / 'data' / 'kitti'
save_path = pcdet_root_dir / 'data' / 'kitti'

<ipython-input-4-e660d07e3510>:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  dataset_cfg = EasyDict(yaml.load(open(dataset_cfg_dir)))


In [5]:
kitti = KittiDataset(dataset_cfg=dataset_cfg, class_names=class_names, root_path=data_path)

In [6]:
infos = kitti.get_infos();

/home/farzad/OpenPCDet/pcdet/utils/calibration_kitti.py:82: RuntimeWarning: divide by zero encountered in true_divide
  pts_img = (pts_2d_hom[:, 0:2].T / pts_rect_hom[:, 2]).T  # (N, 2)


train sample_idx: 000000
train sample_idx: 000001
train sample_idx: 000002
train sample_idx: 000003
train sample_idx: 000004
train sample_idx: 000005
train sample_idx: 000006
train sample_idx: 000007
train sample_idx: 000008
train sample_idx: 000009
train sample_idx: 000010
train sample_idx: 000011
train sample_idx: 000012
train sample_idx: 000013
train sample_idx: 000014
train sample_idx: 000015
train sample_idx: 000016
train sample_idx: 000017
train sample_idx: 000018
train sample_idx: 000019
train sample_idx: 000020
train sample_idx: 000021
train sample_idx: 000022
train sample_idx: 000023
train sample_idx: 000024
train sample_idx: 000025
train sample_idx: 000026
train sample_idx: 000027
train sample_idx: 000028
train sample_idx: 000029
train sample_idx: 000030
train sample_idx: 000031
train sample_idx: 000032train sample_idx: 000033

train sample_idx: 000034
train sample_idx: 000035
train sample_idx: 000036
train sample_idx: 000037
train sample_idx: 000038
train sample_idx: 000039


In [7]:
per_img_infos = []
per_obj_infos = []

for info in tqdm(infos):
    
    annos = info['annos']
    num_gts = annos['gt_boxes_lidar'].shape[0]
    
    lidar_idx = info['point_cloud']['lidar_idx']
    
    img_info = {}
    img_info['lidar_idx'] = lidar_idx
    img_info['num_gts'] = num_gts
    img_info['img_height'] = info['image']['image_shape'][0]
    img_info['img_width'] = info['image']['image_shape'][1]
        
    for i in range(num_gts):
        obj_info = {}
        obj_info['lidar_idx'] = lidar_idx
        
        
        for key, val in annos.items():
            if val.ndim == 2:  # split columns with 2D elements
                for j in range(val.shape[-1]):
                    new_key = key + f'_{j}'
                    obj_info[new_key] = val[i, j]
            else:
                obj_info[key] = val[i]
            
        per_obj_infos.append(obj_info)
    
    per_img_infos.append(img_info)

100%|██████████| 79/79 [00:00<00:00, 9936.72it/s]


In [8]:
per_img_infos_pd = pd.DataFrame.from_dict(per_img_infos)
per_obj_infos_pd = pd.DataFrame.from_dict(per_obj_infos)
report_img = pandas_profiling.ProfileReport(per_img_infos_pd, minimal=True)
report_obj = pandas_profiling.ProfileReport(per_obj_infos_pd, minimal=True)

report_img.to_file("./dataset_reports/carla_kitti_0_9_10_per_img.html")
report_obj.to_file("./dataset_reports/carla_kitti_0_9_10_per_obj.html")